<a href="https://colab.research.google.com/github/rdkdaniel/NLP-Projects/blob/main/English_to_Spanish_translator_with_S_2_S_Transfomer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>